In [32]:
import os
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import FeatureLocation, SimpleLocation, CompoundLocation, ExactPosition, SeqFeature 

def adjust_and_truncate_location(original_location, offset_int, new_seq_length):
    """
    Manually adjusts a FeatureLocation (Simple or Compound) by an integer offset
    and truncates it to fit within [0, new_seq_length) boundaries.
    Returns None if the location becomes invalid (zero-length or outside bounds) after adjustment.
    """
    if original_location is None or not isinstance(original_location, FeatureLocation):
        return None

    original_start_int = int(original_location.start)
    original_end_int = int(original_location.end)

    if isinstance(original_location, SimpleLocation):
        shifted_start = original_start_int - offset_int
        shifted_end = original_end_int - offset_int
        
        truncated_start = max(0, shifted_start)
        truncated_end = min(new_seq_length, shifted_end)

        if truncated_start >= truncated_end:
            return None # Location is now zero-length or invalid after truncation
        
        return SimpleLocation(truncated_start, truncated_end, original_location.strand)
    
    elif isinstance(original_location, CompoundLocation):
        adjusted_parts = []
        for part in original_location.parts:
            adjusted_part = adjust_and_truncate_location(part, offset_int, new_seq_length)
            if adjusted_part is not None:
                adjusted_parts.append(adjusted_part)
        
        if not adjusted_parts:
             return None # No valid parts remaining after adjustment/truncation
        
        return CompoundLocation(adjusted_parts, original_location.strand, original_location.operator)
    
    else:
        return None # Cannot handle unknown location types, treat as invalid


def split_gbk_by_explicit_region(input_gbk_file, output_base_dir, region_feature_types, 
                                   output_format_extension, min_region_length): 
    """
    Splits a single GenBank file into individual region files (e.g., gene clusters).
    Creates a subdirectory for output from each input file.
    """
    if not os.path.exists(output_base_dir):
        os.makedirs(output_base_dir)

    base_name_without_ext = os.path.splitext(os.path.basename(input_gbk_file))[0]
    specific_output_subdir = output_base_dir
    if not os.path.exists(specific_output_subdir):
        os.makedirs(specific_output_subdir)

    region_candidate_count = 0
    written_files_count = 0

    print(f"\n--- Processing file: {os.path.basename(input_gbk_file)} ---")
    print(f"  Outputting to: {specific_output_subdir}")
    print(f"  Looking for feature types: {', '.join(region_feature_types)}")
    print(f"  Minimum region length: {min_region_length} bp")

    try:
        with open(input_gbk_file, "r") as infile:
            for record in SeqIO.parse(infile, "genbank"):
                print(f"  Processing record: {record.id} (Length: {len(record.seq)} bp)")

                if 'molecule_type' not in record.annotations:
                    record.annotations['molecule_type'] = 'DNA'
                    print(f"    Warning: Added 'molecule_type: DNA' to record {record.id} annotations for extraction.")

                valid_original_features = [f for f in record.features if f.location is not None]

                for feature in valid_original_features: # This 'feature' is the BGC region
                    if feature.type in region_feature_types:
                        region_candidate_count += 1
                        
                        if len(feature.location) < min_region_length:
                            cluster_id_for_print = feature.qualifiers.get('locus_tag', ['unknown'])[0]
                            print(f"    Skipping small feature '{cluster_id_for_print}' (type: {feature.type}, length: {len(feature.location)} bp).")
                            continue

                        cluster_id_base = f"cluster_{region_candidate_count}"
                        if 'region_number' in feature.qualifiers:
                             cluster_id_base = f"region_{feature.qualifiers['region_number'][0]}"
                        elif 'candidate_cluster_number' in feature.qualifiers:
                             cluster_id_base = f"candcluster_{feature.qualifiers['candidate_cluster_number'][0]}"
                        elif 'protocluster_number' in feature.qualifiers:
                             cluster_id_base = f"protocluster_{feature.qualifiers['protocluster_number'][0]}"
                        
                        cluster_id = f"{record.id.replace('.', '_')}_{cluster_id_base}"
                        
                        cluster_note = feature.qualifiers.get('note', feature.qualifiers.get('label', ['unspecified region']))[0]
                        cluster_product = feature.qualifiers.get('product', ['unknown cluster type'])[0]

                        filename_base = "".join(c for c in cluster_id if c.isalnum() or c in ('_', '-'))
                        output_file_path = os.path.join(specific_output_subdir, f"{filename_base}.{output_format_extension}")

                        output_biopython_format = "genbank" if output_format_extension == 'gbk' else "fasta"
                        if output_format_extension not in ['gbk', 'fasta']:
                             print(f"    Warning: Unknown output_format_extension '{output_format_extension}'. Defaulting to 'genbank'.")

                        try:
                            cluster_seq = feature.extract(record.seq)
                            if not cluster_seq:
                                raise ValueError(f"Extracted sequence for {cluster_id} is empty (length 0).")

                            cluster_start_int = int(feature.location.start) # Base for shifting
                            new_cluster_seq_length = len(cluster_seq) # Length of this new segment

                            new_record = SeqRecord(
                                cluster_seq,
                                id=f"{record.id}_{cluster_id_base}",
                                name=cluster_id_base[:16],
                                description=f"{cluster_note} | {cluster_product} | {record.id} region {cluster_start_int}-{int(feature.location.end)}"
                            )
                            if 'molecule_type' in record.annotations:
                                new_record.annotations['molecule_type'] = record.annotations['molecule_type']
                            if 'comment' in record.annotations:
                                new_record.annotations['comment'] = record.annotations['comment']

                            # --- NEW: Create a new 'source' feature matching the new record's length ---
                            new_source_feature_location = FeatureLocation(0, new_cluster_seq_length, 1) # 0-based, forward strand default
                            source_qualifiers = {
                                'mol_type': [record.annotations.get('molecule_type', 'DNA')],
                                'organism': record.annotations.get('organism', ['unknown organism']),
                                'strain': record.annotations.get('strain', ['unknown strain'])
                            }
                            original_source_feature = next((f for f in record.features if f.type == 'source'), None)
                            if original_source_feature and 'db_xref' in original_source_feature.qualifiers:
                                source_qualifiers['db_xref'] = original_source_feature.qualifiers['db_xref']

                            new_record.features.append(
                                SeqFeature(new_source_feature_location, type='source', qualifiers=source_qualifiers)
                            )
                            # --- END NEW: Custom source feature ---

                            # Process features to be added to the new record
                            added_gene_cds_locations = set() # To track (start, end, strand) for genes/CDS already processed
                            
                            for internal_feature in valid_original_features:
                                if internal_feature.type == 'source': # Skip original source, a new one was created
                                    continue

                                internal_start_original = int(internal_feature.location.start)
                                internal_end_original = int(internal_feature.location.end)

                                if (internal_start_original < int(feature.location.end) and internal_end_original > cluster_start_int) and \
                                   internal_feature.type not in region_feature_types: # Do not re-add the main region itself
                                    
                                    feature_to_add_type = internal_feature.type
                                    feature_to_add_location = internal_feature.location
                                    feature_to_add_qualifiers = {**internal_feature.qualifiers}

                                    current_feature_loc_tuple = (internal_feature.location.start, internal_feature.location.end, internal_feature.location.strand)

                                    if internal_feature.type == 'CDS':
                                        # Aggressive CDS filtering
                                        missing_qualifiers = []
                                        if 'translation' not in feature_to_add_qualifiers or not feature_to_add_qualifiers['translation'][0].strip():
                                            missing_qualifiers.append('translation (missing or empty)')
                                        elif len(feature_to_add_qualifiers['translation'][0].strip()) < 5:
                                            missing_qualifiers.append(f"translation (too short: {len(feature_to_add_qualifiers['translation'][0].strip())} aa)")
                                        if 'locus_tag' not in feature_to_add_qualifiers or not feature_to_add_qualifiers['locus_tag'][0].strip():
                                            missing_qualifiers.append('locus_tag (missing or empty)')
                                        if 'product' not in feature_to_add_qualifiers or not feature_to_add_qualifiers['product'][0].strip():
                                            missing_qualifiers.append('product (missing or empty)')
                                        
                                        if missing_qualifiers:
                                            print(f"        Warning: Skipping CDS feature '{feature_to_add_qualifiers.get('locus_tag', ['unknown'])[0]}' (in original record {record.id}, location: {internal_feature.location}) due to missing/malformed essential qualifiers: {', '.join(missing_qualifiers)}.")
                                            continue # Skip this malformed CDS feature
                                        
                                        added_gene_cds_locations.add(current_feature_loc_tuple) # Mark this location as having a valid CDS

                                    elif internal_feature.type == 'gene':
                                        # If a CDS for this gene's location was already added, skip this gene to avoid redundancy
                                        if current_feature_loc_tuple in added_gene_cds_locations:
                                            continue

                                        # --- NEW: Create synthetic CDS for 'gene' features ---
                                        # BiG-SCAPE needs CDS features. If only a 'gene' is present, convert it to a CDS.
                                        
                                        try:
                                            gene_dna_seq = internal_feature.extract(record.seq)
                                            if len(gene_dna_seq) % 3 != 0:
                                                gene_dna_seq = gene_dna_seq[:len(gene_dna_seq) - (len(gene_dna_seq) % 3)]
                                            
                                            translated_aa_seq = str(gene_dna_seq.translate(table=11, cds=False))
                                            if translated_aa_seq.endswith('*'):
                                                translated_aa_seq = translated_aa_seq[:-1]

                                            if not translated_aa_seq.strip() or len(translated_aa_seq.strip()) < 5:
                                                print(f"        Warning: Generated translation for gene '{feature_to_add_qualifiers.get('gene', ['unknown'])[0]}' is too short or empty. Skipping gene.")
                                                continue
                                            
                                            feature_to_add_qualifiers['translation'] = [translated_aa_seq]
                                            feature_to_add_qualifiers['codon_start'] = ['1'] # 1 is standard if no partial start.
                                            print(f"        Info: Generated translation for gene '{feature_to_add_qualifiers.get('gene', ['unknown'])[0]}' (length {len(translated_aa_seq)} aa).")

                                        except Exception as e:
                                            print(f"        Warning: Could not translate gene '{feature_to_add_qualifiers.get('gene', ['unknown'])[0]}' (location: {internal_feature.location}): {e}. Skipping gene.")
                                            continue

                                        if 'locus_tag' not in feature_to_add_qualifiers:
                                            gene_locus_tag = feature_to_add_qualifiers.get('gene', [f"gene_{internal_feature.location.start}"])[0]
                                            feature_to_add_qualifiers['locus_tag'] = [gene_locus_tag]
                                        
                                        feature_to_add_type = 'CDS' # Change type to CDS
                                        added_gene_cds_locations.add(current_feature_loc_tuple) # Mark this location as having a valid CDS

                                    else: # Skip other non-CDS/non-gene internal features
                                        continue
                                    # --- END NEW: Synthetic CDS for 'gene' features ---
                                    
                                    # If a feature (either original CDS or synthetic CDS) is ready to be added
                                    adjusted_location = adjust_and_truncate_location(feature_to_add_location, cluster_start_int, new_cluster_seq_length)
                                    
                                    if adjusted_location is None: # Location became invalid after adjustment/truncation
                                        print(f"        Warning: Skipping malformed internal feature '{feature_to_add_type}' (original location: {feature_to_add_location}) as its location became invalid after adjustment/truncation.")
                                        continue

                                    adjusted_feature = internal_feature.__class__( # Use original class type to maintain consistency
                                        adjusted_location,
                                        type=feature_to_add_type, # Use the potentially modified type (CDS for synthetic)
                                        qualifiers=feature_to_add_qualifiers # Use the potentially modified qualifiers
                                    )
                                    new_record.features.append(adjusted_feature)
                            
                            # Add the encompassing region feature itself, RETAINING its original type
                            adjusted_region_location = adjust_and_truncate_location(feature.location, cluster_start_int, new_cluster_seq_length)
                            
                            if adjusted_region_location is None: # Critical check for the main region feature
                                raise ValueError(f"Region feature '{cluster_id}' location became invalid after adjustment/truncation. Cannot create file.")

                            feature_qualifiers_for_adjusted_feature = {**feature.qualifiers}
                            if feature.type == 'region' and 'locus_tag' not in feature_qualifiers_for_adjusted_feature:
                                feature_qualifiers_for_adjusted_feature['locus_tag'] = [cluster_id_base]

                            region_feature_adjusted = feature.__class__(
                                adjusted_region_location,
                                type=feature.type,
                                qualifiers=feature_qualifiers_for_adjusted_feature
                            )
                            new_record.features.insert(0, region_feature_adjusted) # Insert first to be prominent

                            with open(output_file_path, "w") as outfile:
                                SeqIO.write(new_record, outfile, output_biopython_format)
                            print(f"    Wrote {output_biopython_format.capitalize()} for cluster '{cluster_id_base}' (type: {feature.type}) to {os.path.basename(output_file_path)}")
                            written_files_count += 1

                        except ValueError as e:
                            print(f"    Could not write file for region '{cluster_id}' (location {feature.location}): {e}")
                            continue

    except FileNotFoundError:
        print(f"Error: Input file '{input_gbk_file}' not found.")
    except Exception as e:
        print(f"An unexpected error occurred while processing '{input_gbk_file}': {e}")
    
    print(f"  Finished processing {os.path.basename(input_gbk_file)}. Identified {region_candidate_count} features matching specified types. Successfully wrote {written_files_count} files after filtering by length.")


if __name__ == "__main__":
    current_directory = os.getcwd()
    output_base_dir = "split_gbk_regions_output"

    if not os.path.exists(output_base_dir):
        os.makedirs(output_base_dir)

    gbk_files_found = []
    for filename in os.listdir(current_directory):
        if filename.endswith(".gbk"):
            gbk_files_found.append(os.path.join(current_directory, filename))

    if not gbk_files_found:
        print(f"No .gbk files found in the current directory: {current_directory}")
    else:
        print(f"Found {len(gbk_files_found)} .gbk file(s) in '{current_directory}'.")
        print(f"All extracted regions will be placed in subdirectories within '{output_base_dir}'.")
        print("-" * 50)

        for gbk_file_path in gbk_files_found:
            split_gbk_by_explicit_region(
                gbk_file_path,
                output_base_dir,
                region_feature_types=['region'],
                output_format_extension='gbk',
                min_region_length=1000
            )
            print("-" * 50)

        print(f"\nAll specified GenBank files processed. Output can be found in '{output_base_dir}'.")

Found 11 .gbk file(s) in 'c:\Users\poker\Downloads'.
All extracted regions will be placed in subdirectories within 'split_gbk_regions_output'.
--------------------------------------------------

--- Processing file: GCF_000154965.1.gbk ---
  Outputting to: split_gbk_regions_output
  Looking for feature types: region
  Minimum region length: 1000 bp
  Processing record: NZ_CM000951.1 (Length: 9313494 bp)
        Info: Generated translation for gene 'unknown' (length 475 aa).
        Info: Generated translation for gene 'unknown' (length 522 aa).
        Info: Generated translation for gene 'unknown' (length 62 aa).
        Info: Generated translation for gene 'unknown' (length 407 aa).
        Info: Generated translation for gene 'unknown' (length 247 aa).
        Info: Generated translation for gene 'unknown' (length 179 aa).
        Info: Generated translation for gene 'unknown' (length 424 aa).
        Info: Generated translation for gene 'unknown' (length 200 aa).
        Info: Gene